# Run the `Heat` model through its BMI

`Heat` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. View the source code for the [model](https://github.com/csdms/bmi-example-python/blob/master/heat/heat.py) and its [BMI](https://github.com/csdms/bmi-example-python/blob/master/heat/bmi_heat.py) on GitHub.

Start by importing `os`, `numpy` and the `Heat` BMI:

In [1]:
import os
import numpy as np

from heat import BmiHeat

Create an instance of the model's BMI.

In [2]:
x = BmiHeat()

What's the name of this model?

In [3]:
print(x.get_component_name())

The 2D Heat Equation


Start the `Heat` model through its BMI using a configuration file:

In [4]:
cat heat.yaml

# Heat model configuration
shape:
  - 6
  - 8
spacing:
  - 1.0
  - 1.0
origin:
  - 0.0
  - 0.0
alpha: 1.0


In [5]:
x.initialize("heat.yaml")

Check the time information for the model.

In [28]:
stateOut = x.get_state()
print(stateOut)

{
    "time": 0.0,
    "plate_surface__temperature": [
        0.9684836247379001,
        0.6550080046844243,
        0.5254915600187879,
        0.619005434495118,
        0.663366876795297,
        0.008368857864502366,
        0.4851313825711857,
        0.8498053312201644,
        0.524668279750368,
        0.1747861537340425,
        0.06985965171692354,
        0.3376524402569979,
        0.7057232571227028,
        0.8376753702730808,
        0.756051697274303,
        0.6964356289755126,
        0.3346446608257382,
        0.18133878239070256,
        0.041532354468550525,
        0.7412339094649948,
        0.35970724575029756,
        0.19669693152150403,
        0.1910863473417963,
        0.1740324927025939,
        0.15983348466606206,
        0.16192261102228234,
        0.6609493865002737,
        0.268052042681284,
        0.3538134291823436,
        0.9953041956155207,
        0.8072033397281079,
        0.7442212040297197,
        0.4967462109320222,
        0.405104

In [16]:
import json
stateOutDict = json.loads(stateOut)

In [25]:
stateOutDict['plate_surface__temperature'][15]=0
print(stateOutDict)

{'time': 0.0, 'plate_surface__temperature': [0.9684836247379001, 0.6550080046844243, 0.5254915600187879, 0.619005434495118, 0.663366876795297, 0.008368857864502366, 0.4851313825711857, 0.8498053312201644, 0.524668279750368, 0.1747861537340425, 0.06985965171692354, 0.3376524402569979, 0.7057232571227028, 0.8376753702730808, 0.756051697274303, 0, 0.3346446608257382, 0.18133878239070256, 0.041532354468550525, 0.7412339094649948, 0.35970724575029756, 0.19669693152150403, 0.1910863473417963, 0.1740324927025939, 0.15983348466606206, 0.16192261102228234, 0.6609493865002737, 0.268052042681284, 0.3538134291823436, 0.9953041956155207, 0.8072033397281079, 0.7442212040297197, 0.4967462109320222, 0.40510407107593627, 0.310218753116829, 0.7598591849829671, 0.7619716855689805, 0.7099200788917801, 0.5353736828614781, 0.7000529188078973, 0.21201273833116563, 0.4263668206378457, 0.3152553684510294, 0.9059135080756453, 0.39613599806447175, 0.7721687861939127, 0.8618269496624634, 0.3039643569841276]}


In [29]:
x.set_state(json.dumps(stateOutDict))

In [30]:
print("Start time:", x.get_start_time())
print("End time:", x.get_end_time())
print("Current time:", x.get_current_time())
print("Time step:", x.get_time_step())
print("Time units:", x.get_time_units())

Start time: 0.0
End time: 1.7976931348623157e+308
Current time: 0.0
Time step: 0.25
Time units: s


Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [31]:
print(x.get_input_var_names())
print(x.get_output_var_names())

('plate_surface__temperature',)
('plate_surface__temperature',)


Next, get the identifier for the grid on which the temperature variable is defined:

In [32]:
grid_id = x.get_var_grid("plate_surface__temperature")
print("Grid id:", grid_id)

Grid id: 0


Then get the grid attributes:

In [33]:
print("Grid type:", x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print("Grid rank:", rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print("Grid shape:", shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print("Grid spacing:", spacing)

Grid type: uniform_rectilinear
Grid rank: 2
Grid shape: [6 8]
Grid spacing: [1. 1.]


These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [34]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value("plate_surface__temperature", temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [35]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

[[  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]


Now advance the model by a single time step:

In [ ]:
x.update()

View the new state of the temperature field:

In [ ]:
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

There's diffusion!

Advance the model to some distant time:

In [ ]:
distant_time = 2.0
while x.get_current_time() < distant_time:
    x.update()

View the final state of the temperature field:

In [ ]:
np.set_printoptions(formatter={"float": "{: 5.1f}".format})
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Note that temperature isn't conserved on the plate:

In [ ]:
print(temperature_flat.sum())

End the model:

In [ ]:
x.finalize()